In [1]:
'''update the config.yaml file
the constants will be unchanged
update the entity
update the configuration manager
do the test train split
'''
from dataclasses import dataclass
from pathlib import Path
@dataclass
class DataTransformationConfig:
    raw_data:Path
    schema_data:Path
    transf_train_set:Path# i need where to store my tranformed train set after splitting,it will be pointing to artifacts.datatransformation folder
    transf_test_set:Path



    

    

In [2]:
from src.mlproject.constants import CONFIG_PATH_YAML,SCHEMA_PATH_YAML
from src.mlproject.entity.config_entity import DataIngestionConfig
from src.mlproject.utils.common import read_yaml,create_directories
import os


class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_PATH_YAML) :#inner folder,pathto csv,datastore
        self.config_data=read_yaml(config_filepath)
        self.schema=read_yaml(SCHEMA_PATH_YAML)
        '''
         #Retrieve the parent folder 
        self.parent_folder =self.config_data.get('parent_folder', '')
         #Retrieve the artifcats root 
        self.artifacts_root =self.parent_folder.get('artifacts_root', '')
        #and create the folder 'artifacts_root'
        create_directories([self.artifacts_root])'''

        
         #DATA TRANSFORMATION
        #retrieve the data ttransformation category
        self.data_transformation=self.config_data.get('data_transformation','')
        #create the folder within it 
        create_directories([self.data_transformation.get('inner_folder','')])
        #join the files
        # join the files and assign the result to variables
        self.transformed_train_set_path = os.path.join(self.data_transformation.get('inner_folder', ''),self.data_transformation.get('transformed_train_set', ''))
        #os.path.join(self.data_transormation.get('inner_folder',''),self.data_transormation.get('transformed_train_set',''))
        self.transformed_test_set_path=os.path.join(self.data_transformation.get('inner_folder',''),self.data_transformation.get('transformed_test_set',''))
        
       
       
        
         
       #instantiate the defined entities and return the instance
    def get_data_transformation_config(self)->DataTransformationConfig:

            data_transformation_config = DataTransformationConfig(
                transf_train_set=self.transformed_train_set_path,
                transf_test_set=self.transformed_test_set_path,
                raw_data=self.data_transformation.get('raw_data',''),
                schema_data=self.schema
                
                
                )
            
            return data_transformation_config
    
    
                
            

        



In [3]:
'''
define the categorical columns and numerical columns and the target column
split into train and test
define the train x and train y,test x and test y
define the transformation to be performed on categorical 
define the transformation to be performed on numerical 
save the transformation object
combine the trainx and train y
'''
from sklearn.model_selection import train_test_split
from src.mlproject.utils.common import read_yaml_keys
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
#from scipy.sparse import issparse

from sklearn.preprocessing import MaxAbsScaler

class DataTransformationEntity:
    def __init__(self,config_instance:DataTransformationConfig) :
        self.transf_config=config_instance
       
        
        self.test_set_transf=self.transf_config.transf_test_set
        self.train_set_transf=self.transf_config.transf_train_set
        self.raw_data=self.transf_config.raw_data
        self.schemapath=self.transf_config.schema_data

    '''def accesss_trans_config_elements(self):
        self.transf_train_set=self.transf_config.transf_test_set
        self.trans_test_set=self.transf_config.transf_test_set'''
    def initiate_split(self):
        categorical_col,numerical_cal,target_col=read_yaml_keys(self.transf_config.schema_data)
        
        raw_data_df=pd.read_csv(self.raw_data)

        

        train,test=train_test_split(raw_data_df,random_state=42,test_size=0.2)

        

        train_df=pd.DataFrame(train,columns=raw_data_df.columns)
        test_df=pd.DataFrame(test,columns=raw_data_df.columns)

        
        train_y=train_df[target_col]
        
        
        train_x = train_df.drop(target_col, axis=1)
        

        test_y=test_df[target_col]
        test_x=test_df.drop(target_col,axis=1)
       
        
        return train_x,train_y,test_x,test_y
    def transformations(self):
        cat_pipeline=Pipeline(
            steps=[
            #('imputer',SimpleImputer(strategy="most_frequent")),
            ('encoder',OneHotEncoder(handle_unknown='ignore')),
            ]
            )
        '''num_pipeline=Pipeline(
            steps=[
            ('impute',SimpleImputer(strategy='median')),
            ('scaler',StandardScaler(with_mean=False))
            ]
        )'''


        num_pipeline = Pipeline(
           steps=[
          # ('impute', SimpleImputer(strategy='median')),
           ('scaler', StandardScaler())
    ]
)
        return cat_pipeline,num_pipeline
    

    
  
    def cols_transformer(self):
        categorical_col,numerical_cal,target_col=read_yaml_keys(self.transf_config.schema_data)
        train_x,train_y,test_x,test_y=self.initiate_split()

        cat_pipe,num_pipe=self.transformations()
        
        preprocessing_obj=ColumnTransformer(transformers=[('num',num_pipe,numerical_cal),('cat',cat_pipe,categorical_col)],remainder='drop')
        # Assuming preprocessing_obj is the ColumnTransformer
        
        
        
        return preprocessing_obj

In [4]:
'''the actual transformation
 as it has all the transformations

pass in the entities you need from the the DataTransfofrmationEntity,an instance of the DataTransformationEntity
 
do the actujal transformation using entities from DataTransfofrmationEntity
store the transformations in the paths specified by DataTransformationConfig instnce which i put as attributtes of the DataTransformationEntity
TRANSFORMATION>PY'''
import numpy as np
import pandas as pd
class DataTransformation:
    def __init__(self,configEnt:DataTransformationEntity) :
        
        self.transformation_entity=configEnt#an ins
        
        
    def transformation(self):
        categorical_col,numerical_cal,target_col=read_yaml_keys(self.transformation_entity.schemapath)

        train_x_transf, train_y, test_x_transf,test_y=self.transformation_entity.initiate_split()
        print(test_y)
        
        self.transformation_entity.transformations()
        prep_obj=self.transformation_entity.cols_transformer()

        
        trainx_transformed_data=prep_obj.fit_transform(train_x_transf)
        
        # Get feature names after transformation
        transformed_feature_names = prep_obj.get_feature_names_out(input_features=categorical_col+ numerical_cal)

       # Create a DataFrame with the transformed data and feature names
        trainx_transformed_df = pd.DataFrame(trainx_transformed_data, columns=transformed_feature_names)
        trainx_transformed_df[target_col]=train_y.values# if forgot the .values so it really was a prob
        

        
       
        testx_transformed_data=prep_obj.transform(test_x_transf)
        testx_transformed_df = pd.DataFrame(testx_transformed_data, columns=transformed_feature_names)
        testx_transformed_df[target_col] = test_y.values  # Use .values to get the underlying NumPy array, if you ommit,you  get values assigned to some records not all
        print(testx_transformed_df.head(1))
        print(testx_transformed_df.shape)
        
    

       
        with open(self.transformation_entity.train_set_transf,'w') as train_csv:
           trainx_transformed_df.to_csv(train_csv,index=False)

        with open(self.transformation_entity.test_set_transf,'w') as test_csv:
            testx_transformed_df.to_csv(test_csv,index=False)
        

In [5]:
'''pipeline'''

class TransformationPipeline():
    def __init__(self) -> None:
        pass

    def main(self):
        configmanager=ConfigurationManager()
        data_config_instance=configmanager.get_data_transformation_config()
        entity_instance=DataTransformationEntity(data_config_instance)
        data_transf_instance=DataTransformation(entity_instance)
        data_transf_instance.transformation()
     
if __name__=='__main__':
   obj=TransformationPipeline()
   obj.main()
     

521    91
737    53
740    80
660    74
411    84
       ..
408    52
332    62
208    74
613    65
78     61
Name: math_score, Length: 200, dtype: int64
   num__reading_score  num__writing_score  cat__gender_female  \
0            1.137866            1.031032                 1.0   

   cat__gender_male  cat__race_ethnicity_group A  cat__race_ethnicity_group B  \
0               0.0                          0.0                          0.0   

   cat__race_ethnicity_group C  cat__race_ethnicity_group D  \
0                          1.0                          0.0   

   cat__race_ethnicity_group E  \
0                          0.0   

   cat__parental_level_of_education_associate's degree  \
0                                                1.0     

   cat__parental_level_of_education_bachelor's degree  \
0                                                0.0    

   cat__parental_level_of_education_high school  \
0                                           0.0   

   cat__parental_leve